## Import and preprocessing

In [3]:
# imports for preprocessing
import numpy as np
import nltk
import csv
# importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from collections import Counter
# from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import WordPunctTokenizer
import regex

# imports for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, BertModel, BertTokenizerFast
from sklearn.decomposition import PCA
from collections import defaultdict
import wandb
from prettytable import PrettyTable

# imports for modeling
import torch
from transformers import Trainer, TrainingArguments
import pandas as pd
from sklearn.metrics import f1_score
from transformers import BertForSequenceClassification, BertTokenizer, AutoConfig ,TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import optuna
import os
from pathlib import Path


# imports for pruning
import copy
import torch.nn.utils.prune as prune
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\levan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
home_dir = os.getcwd()
output_dir = Path(home_dir, 'results')

In [5]:
home_dir

'c:\\Users\\levan\\Documents\\מסמכים של ניר\\אוניברסיטת תל אביב\\2022-2023\\נושאים מתקדמים בלמידה עמוקה\\פרויקט\\news-dl-project'

## Prepare the data

In [6]:
balanced_df = pd.read_csv('balanced_df.csv')

In [7]:
# if we run on cpu
# balanced_df = balanced_df.iloc[:100]

In [9]:
headlines = balanced_df['text']
categories = balanced_df['updated_category']

# encode the labels to number
label_encoder = LabelEncoder()
balanced_df['label'] = label_encoder.fit_transform(categories.values)

In [10]:
X = headlines.values
y = balanced_df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_df = pd.DataFrame({'headlines':X_train, 'labels':y_train})
test_df = pd.DataFrame({'headlines':X_test, 'labels':y_test})

train_df.to_csv('train_df.csv', index = False)
test_df.to_csv('test_df.csv', index = False)

data_files = {
'train':'train_df.csv',
'test':'test_df.csv'
}

datasets = load_dataset("csv", data_files=data_files)

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 40.00it/s]


Dataset csv downloaded and prepared to C:/Users/levan/.cache/huggingface/datasets/csv/default-7b22cbae0e9605fe/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 32.78it/s]


In [11]:
datasets

DatasetDict({
    train: Dataset({
        features: ['headlines', 'labels'],
        num_rows: 148771
    })
    test: Dataset({
        features: ['headlines', 'labels'],
        num_rows: 37193
    })
})

## Define the model

In [12]:
class Model():
    def __init__(self, model_name, data_set, num_classes):
        self.model_name=model_name
        self.num_classes = num_classes
        self.model= BertForSequenceClassification.from_pretrained(self.model_name, num_labels=self.num_classes, return_dict=True).to(device)
        self.dataset=data_set


    def tokenize(self, token_args):
        self.tokenizer =  BertTokenizer.from_pretrained(self.model_name)
        self.tokenized_dataset = self.dataset.map(self.tokenizer, input_columns='headlines', fn_kwargs=token_args)
        self.tokenized_dataset.set_format('torch')

    def metric_fn(self, predictions):
        preds = predictions.predictions.argmax(axis=1)
        labels = predictions.label_ids
        return {'f1': f1_score(labels, preds, average='weighted')}


    def train(self, train_args):
        trainer = Trainer(
            model=self.model,
            args=train_args,
            train_dataset=self.tokenized_dataset['train'],
            eval_dataset=self.tokenized_dataset['test'],
            compute_metrics=self.metric_fn)

        trainer.train()


    def hyper_parameters_search(self, train_args):
        trainer = Trainer(
            model=self.model,
            args=train_args,
            train_dataset=self.tokenized_dataset['train'],
            eval_dataset=self.tokenized_dataset['test'],
            model_init=self.model_init,
            compute_metrics=self.metric_fn)

        best_run = trainer.hyperparameter_search(direction="maximize", hp_space=self.optuna_hp_space,n_trials=10)
        chosen_hyperparameters = best_run.hyperparameters
        wandb.finish()
        print(f'{self.model_name} chosen hyperparameters:')
        print(chosen_hyperparameters)


    def model_init(self):
        return self.model

    def optuna_hp_space(self,trial):
        return {"learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
                "num_train_epochs": trial.suggest_categorical("num_train_epochs", [7]),
                "seed": trial.suggest_categorical("seed", [9]),
                "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16]),
                "gradient_accumulation_steps":trial.suggest_int("gradient_accumulation_steps",1,5),
                "warmup_steps":trial.suggest_int("warmup_steps",1,300),
                "weight_decay":trial.suggest_float("weight_decay",1e-3,1e-1),
                "per_device_eval_batch_size":trial.suggest_categorical("per_device_eval_batch_size",[16])}


    def print_model(self):
        x = PrettyTable()
        x.field_names  = ["Layer Name", "Sum of Weights"]
        total_sum = 0
        for name, module in self.model.named_modules():
            if isinstance(module, torch.nn.Linear):
                x.add_row([name, round(module.weight.data.abs().sum().item(),3)])
                total_sum+=module.weight.data.abs().sum().item()
        x.add_row(['Total Sum of Weights', round(total_sum,3)])
        print(x)

    def print_model_size(self):
        torch.save(self.model.state_dict(), Path(home_dir,"tmp.pt"))
        print("%.2f MB" %(os.path.getsize(Path(home_dir,"tmp.pt"))/1e6))

    def pruning(self, amount):
        for name, module in self.model.named_modules():
            if isinstance(module,torch.nn.Linear):
                prune.l1_unstructured(module, name='weight', amount=amount)

    def quantize_model(self):
        self.model.qconfig = torch.quantization.default_qconfig
        self.model = torch.quantization.prepare(self.model)
        self.model = torch.quantization.convert(self.model)

    def evaluate(self, train_args):
        trainer = Trainer(
            model=self.model,
            args=train_args,
            train_dataset=self.tokenized_dataset['train'],
            eval_dataset=self.tokenized_dataset['test'],
            compute_metrics=self.metric_fn)

        predictions = trainer.predict(self.tokenized_dataset['test'])
        result_dict = self.metric_fn(predictions)
        for k,v in result_dict.items():
            print(f'{k} value: {v}')

    def kl(self,hidden_dim,num_layers,num_heads,epochs, alpha_teacher, lr, loss_func):
        embedding_matrix = self.model.bert.embeddings.word_embeddings.weight
        embedding_dim = embedding_matrix.size(1)
        vocab_size = embedding_matrix.size(0)
        Student = Student_Calssifier(vocab_size, embedding_dim, hidden_dim, num_layers, num_heads, self.num_classes)
        Student.embedding.weight.data.copy_(embedding_matrix)
        Teacher = copy.deepcopy(self.model).to(device)
        Student.to(device)
        data = self.tokenized_dataset['train']

        Student = kl_training(Student, Teacher, epochs, alpha_teacher, lr, loss_func, data)
        return Student

    def save_model(self, model_name):
        torch.save(self.model.state_dict(), f'{model_name}.pt')



# <u>Model 1 - BERT Base Uncased<u>
## Model 1  Definition and Tokenization:

In [13]:
pip install requests==2.27.1

  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
                                              0.0/143.1 kB ? eta -:--:--
     --------                                30.7/143.1 kB 1.4 MB/s eta 0:00:01
     ------------------                    71.7/143.1 kB 653.6 kB/s eta 0:00:01
     ----------------------------         112.6/143.1 kB 819.2 kB/s eta 0:00:01
     ------------------------------------ 143.1/143.1 kB 851.7 kB/s eta 0:00:00
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.1.0
    Uninstalling charset-normalizer-3.1.0:
      Successfully uninstalled charset-normalizer-3.1.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python310\\Lib\\site-packages\\~harset_normalizer\\md.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [14]:
os.environ['CURL_CA_BUNDLE'] = ''

In [15]:
model_name_1 = "bert-base-uncased"
num_of_classes = 62
bert_model = Model(model_name_1, datasets, num_of_classes)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [16]:
token_args = {"max_length": 64, "truncation": True, "padding": "max_length"}
bert_model.tokenize(token_args)

## Model 1 Hyperparameter Search

In [ ]:
wandb.init(project="DeepLearning")

train_args = TrainingArguments(output_dir=output_dir,
                             overwrite_output_dir=True,
                             greater_is_better=True,
                             evaluation_strategy='epoch',
                             do_train=True,
                             logging_strategy='epoch',
                             save_strategy='no',
                             report_to='wandb')

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.wandb.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.wandb.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
wandb: Currently logged in as: liyag (delta_lxr). Use `wandb login --relogin` to force relogin


In [ ]:
bert_model.hyper_parameters_search(train_args)

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\trainer.py:359: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(
[I 2023-06-12 23:56:45,096] A new study created in memory with name: no-name-5b015cbb-7d7d-4411-8219-748edfad20d2
C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,3.012800,2.324755,0.385335
2,2.162800,1.966571,0.473018
3,1.874100,1.819094,0.513033
4,1.702700,1.740530,0.531638
5,1.582300,1.689133,0.551586
6,1.505800,1.658887,0.560356
7,1.461000,1.650897,0.562540


[I 2023-06-13 01:19:22,168] Trial 0 finished with value: 0.5625395808513218 and parameters: {'learning_rate': 4.183323918010571e-06, 'num_train_epochs': 7, 'seed': 9, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 1, 'warmup_steps': 115, 'weight_decay': 0.08914088722040585, 'per_device_eval_batch_size': 16}. Best is trial 0 with value: 0.5625395808513218.
C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


eval/f1,▁▄▆▇███
eval/loss,█▄▃▂▁▁▁
eval/runtime,▁▇█████
eval/samples_per_second,█▂▁▁▁▁▁
eval/steps_per_second,█▂▁▁▁▁▁
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/learning_rate,█▇▆▅▃▂▁
train/loss,█▄▃▂▂▁▁
train/total_flos,▁
train/train_loss,▁


Epoch,Training Loss,Validation Loss,F1
1,1.527100,1.546883,0.591352
2,1.112200,1.424080,0.635877
3,0.815000,1.397030,0.658872
4,0.600300,1.433538,0.669496
5,0.434200,1.475755,0.679990
6,0.317900,1.537445,0.685465
7,0.243000,1.563465,0.686887


...

eval/f1,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
train/learning_rate,▁
train/loss,▁
eval/f1,0.68628
eval/loss,3.08641


Epoch,Training Loss,Validation Loss,F1
1,0.001700,3.164068,0.683753


[I 2023-06-13 07:17:28,779] Trial 9 pruned. 


eval/f1,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁
train/learning_rate,▁
train/loss,▁
eval/f1,0.68375
eval/loss,3.16407


bert-base-uncased chosen hyperparameters:
{'learning_rate': 1.8325249831728925e-06, 'num_train_epochs': 7, 'seed': 9, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 3, 'warmup_steps': 181, 'weight_decay': 0.014021528948180751, 'per_device_eval_batch_size': 16}


## Model 1 Train on best hyperparameters

In [ ]:
best_args = TrainingArguments(output_dir=output_dir,
                         overwrite_output_dir=True,
                         per_device_train_batch_size=16,
                         per_device_eval_batch_size=16,
                         seed = 9,
                         learning_rate=1.8325249831728925e-06,
                         weight_decay=0.014021528948180751,
                         greater_is_better=True,
                         evaluation_strategy='epoch',
                         do_train=True,
                         num_train_epochs=15,
                         gradient_accumulation_steps=3,
                         logging_strategy='epoch',
                         warmup_steps=181,
                         report_to='wandb')

eval_args = TrainingArguments(output_dir=output_dir,
                         overwrite_output_dir=True,
                         per_device_train_batch_size=16,
                         per_device_eval_batch_size=16,
                          seed = 9,
                         learning_rate=1.8325249831728925e-06,
                         weight_decay=0.014021528948180751,
                         greater_is_better=True,
                         evaluation_strategy='epoch',
                         do_train=False,
                         gradient_accumulation_steps=3,
                         logging_strategy='epoch',
                         warmup_steps=181,
                         report_to='wandb')

bert_model.train(best_args)
# save model's state dict
bert_model.save_model('bert_regular')

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
#bert_model.save_model('bert_regular')
torch.save(bert_model.model.state_dict(),"bert_regular.pt")

## Best model summary:

In [ ]:
bert_model.model.load_state_dict(torch.load(Path(home_dir, 'bert_regular.pt')))
bert_model.evaluate(eval_args)
bert_model.print_model_size()
bert_model.print_model()

f1 value: 0.6869010703006206
438.19 MB
+----------------------------------------------+----------------+
|                  Layer Name                  | Sum of Weights |
+----------------------------------------------+----------------+
|  bert.encoder.layer.0.attention.self.query   |    18635.98    |
|   bert.encoder.layer.0.attention.self.key    |   18326.658    |
|  bert.encoder.layer.0.attention.self.value   |   12523.816    |
| bert.encoder.layer.0.attention.output.dense  |   12118.736    |
|   bert.encoder.layer.0.intermediate.dense    |    65533.0     |
|      bert.encoder.layer.0.output.dense       |   61711.406    |
|  bert.encoder.layer.1.attention.self.query   |   18492.051    |
|   bert.encoder.layer.1.attention.self.key    |   18417.031    |
|  bert.encoder.layer.1.attention.self.value   |   12342.176    |
| bert.encoder.layer.1.attention.output.dense  |   11883.429    |
|   bert.encoder.layer.1.intermediate.dense    |   68518.859    |
|      bert.encoder.layer.1.output.de

## a. Model 1 Pruning

In [ ]:
pruned_bert = Model(model_name_1, datasets, num_of_classes)
pruned_bert.tokenize(token_args)

# Load the model's best parameters
pruned_bert.model.load_state_dict(torch.load(Path(home_dir, 'bert_regular.pt')))

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT

<All keys matched successfully>

In [ ]:
# Prune and train one epoch
pruned_bert.pruning(0.2)
train_one_epoch_args = TrainingArguments(output_dir=output_dir,
                             overwrite_output_dir=True,
                             greater_is_better=True,
                             evaluation_strategy='epoch',
                             do_train=True,
                             logging_strategy='epoch',
                             num_train_epochs=1,
                             save_strategy='no')

pruned_bert.train(train_one_epoch_args)
# save model's state dict
pruned_bert.save_model('bert_pruning')

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.655800,2.059624,0.677904


In [ ]:
# Pruned model summary
pruned_bert.evaluate(eval_args)
pruned_bert.print_model_size()
pruned_bert.print_model()

f1 value: 0.677904232049942
780.50 MB
+----------------------------------------------+----------------+
|                  Layer Name                  | Sum of Weights |
+----------------------------------------------+----------------+
|  bert.encoder.layer.0.attention.self.query   |   18088.371    |
|   bert.encoder.layer.0.attention.self.key    |    17797.49    |
|  bert.encoder.layer.0.attention.self.value   |   12151.385    |
| bert.encoder.layer.0.attention.output.dense  |   11763.169    |
|   bert.encoder.layer.0.intermediate.dense    |   63676.031    |
|      bert.encoder.layer.0.output.dense       |   59927.969    |
|  bert.encoder.layer.1.attention.self.query   |   17953.551    |
|   bert.encoder.layer.1.attention.self.key    |   17884.939    |
|  bert.encoder.layer.1.attention.self.value   |   11963.402    |
| bert.encoder.layer.1.attention.output.dense  |   11522.994    |
|   bert.encoder.layer.1.intermediate.dense    |   66579.664    |
|      bert.encoder.layer.1.output.den

## b. Model 1 Quantization

In [19]:
# Load the quantized model

quantization_bert = Model(model_name_1, datasets, num_of_classes)
quantization_bert.tokenize(token_args)
quantization_bert.model.load_state_dict(torch.load(Path(home_dir, 'bert_regular.pt'), map_location=torch.device(device)))

# Train the quantized model for additional epochs
train_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    num_train_epochs=5,  # Specify the desired number of additional epochs
    save_strategy='no'
)

quantization_bert.train(train_args)

# Save the trained quantized model's state dict
quantization_bert.save_model('quantized_model_trained.pt')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

  0%|          | 56/92985 [07:28<254:17:16,  9.85s/it]

KeyboardInterrupt: 

## c. Model 1 Knowledge-Distillation

## Help function for model distilation

In [ ]:
class Student_Calssifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_heads, num_classes, dropout=0.1):
        super(Student_Calssifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder_layers = nn.TransformerEncoderLayer(embedding_dim, num_heads, hidden_dim, dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layers, num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        embedded = embedded.permute(1, 0, 2)  # Adjust shape for transformer
        transformer_output = self.transformer_encoder(embedded)
        transformer_output = transformer_output.permute(1, 0, 2)  # Adjust shape back to (batch_size, seq_length, embedding_dim)
        logits = self.fc(transformer_output[:, -1, :])  # Take the last hidden state
        return logits.squeeze()


In [ ]:
def kl_training(Student, Teacher, epochs, alpha_teacher, lr, loss_func, data):
    optimizer = torch.optim.Adam(Student.parameters(), lr=lr)
    count = 0
    optimizer.zero_grad()
    losses = list()

    for epoch in range(epochs):
        for index, i in enumerate(data):
            loss_e = 0
            count += 1
            inputs = i['input_ids']
            y_index = i['labels']
            attention_mask = torch.tensor(i['attention_mask']).to(device)
            inputs = torch.tensor(inputs).to(device)
            # Calculate loss with respect to teacher logits
            teacher_logits = Teacher(input_ids=inputs.unsqueeze(-1), attention_mask=attention_mask.unsqueeze(-1))[0]
            outputs = Student(inputs.unsqueeze(-1))
            loss_teacher = loss_func(outputs, teacher_logits)

            # Calculate loss with respect to ground truth
            temp_array = np.zeros(num_classes)
            temp_array[y_index] = 1
            target = torch.from_numpy(temp_array).unsqueeze(0).to(device)
            loss_ground_truth = loss_func(outputs, target)

            loss = alpha_teacher * loss_teacher + (1 - alpha_teacher) * loss_ground_truth
            loss_e += loss.item()
            if count == 64:
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                count = 0
            if index%1000 == 0:
                print(index)
        losses.append(loss_e)
        print(f'Epoch {epoch} was finished with train loss = {loss_e}')
    return Student

In [ ]:
hidden_dim = 128
num_layers = 2
num_heads = 2
num_classes = 62
epochs = 5
alpha_teacher = 0.8
lr=0.001
loss_func = nn.MSELoss()


In [ ]:
kl_bert = Model(model_name_1, datasets, num_of_classes)
kl_bert.tokenize(token_args)

# Load the model's best parameters
kl_bert.model.load_state_dict(torch.load(Path(home_dir, 'bert_regular.pt')))

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT

<All keys matched successfully>

In [ ]:
bert_student = kl_bert.kl(hidden_dim,num_layers,num_heads,epochs, alpha_teacher, lr, loss_func)

C:\Users\liyag\AppData\Local\Temp\ipykernel_21452\2360740035.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(i['attention_mask']).to(device)
C:\Users\liyag\AppData\Local\Temp\ipykernel_21452\2360740035.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs).to(device)
C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1, 62])) that is different to the input size (torch.Size([64, 62])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

0


RuntimeError: Found dtype Double but expected Float

In [ ]:
student_model = Model(model_name_1, datasets, num_of_classes)
student_model.tokenize(token_args)
student_model.model = bert_student

# save model's state dict
student_model.save_model('bert_student')

In [ ]:
# Student model summary
student_model.evaluate(eval_args)
student_model.print_model_size()
student_model.print_model()

# <u>Model 2 - ROBERTA Base<u>
## Model 2 Definition and Tokenization:

In [ ]:
model_name_2 = "roberta-base"
num_of_classes = 62
roberta = Model(model_name_2, datasets, num_of_classes)
token_args = {"max_length": 64, "truncation": True, "padding": "max_length"}
roberta.tokenize(token_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\liyag\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Map:   0%|          | 0/148800 [00:00<?, ? examples/s]

Map:   0%|          | 0/37200 [00:00<?, ? examples/s]

## Model 2 Hyperparameter Search

In [ ]:
wandb.init(project="DeepLearning")

train_args = TrainingArguments(output_dir=output_dir,
                             overwrite_output_dir=True,
                             greater_is_better=True,
                             evaluation_strategy='epoch',
                             do_train=True,
                             logging_strategy='epoch',
                             save_strategy='no',
                             report_to='wandb')



wandb: Currently logged in as: liyag (delta_lxr). Use `wandb login --relogin` to force relogin


In [ ]:
roberta.hyper_parameters_search(train_args)

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\trainer.py:359: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(
[I 2023-06-15 18:24:12,873] A new study created in memory with name: no-name-341e8ab4-bff7-482f-a97a-b3ee3a305697
C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,3.494300,2.952349,0.237611
2,2.867600,2.599500,0.314514
3,2.624600,2.439085,0.354110
4,2.487900,2.346071,0.368190
5,2.401000,2.289408,0.385501
6,2.351900,2.260303,0.389990
7,2.325400,2.249121,0.392198


[I 2023-06-15 19:33:30,864] Trial 0 finished with value: 0.392197507549085 and parameters: {'learning_rate': 1.0906450785722703e-06, 'num_train_epochs': 7, 'seed': 9, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 1, 'warmup_steps': 298, 'weight_decay': 0.02419408513272277, 'per_device_eval_batch_size': 16}. Best is trial 0 with value: 0.392197507549085.
C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


eval/f1,▁▄▆▇███
eval/loss,█▄▃▂▁▁▁
eval/runtime,▂▂▁▅█▅▁
eval/samples_per_second,▇▇█▄▁▄█
eval/steps_per_second,▇▇█▄▁▄█
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/learning_rate,█▇▆▅▃▂▁
train/loss,█▄▃▂▁▁▁
train/total_flos,▁
train/train_loss,▁


Epoch,Training Loss,Validation Loss,F1
1,2.277000,2.171113,0.414297
2,2.167500,2.098306,0.436412
3,2.100200,2.053305,0.450292
4,2.051400,2.018600,0.454959
5,2.013100,1.991603,0.466067
6,1.994400,1.978873,0.467591
7,1.984600,1.972843,0.470023


...

eval/f1,█▄█▆▆▂▁
eval/loss,▁▄▄▆███
eval/runtime,█▆▁▄▆▇▁
eval/samples_per_second,▁▃█▅▃▂█
eval/steps_per_second,▁▃█▅▃▂█
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/learning_rate,█▇▆▅▃▂▁
train/loss,▁▁▁▁▁▁█
train/total_flos,▁
train/train_loss,▁


Epoch,Training Loss,Validation Loss,F1
1,0.000100,3.257234,0.686035
2,0.000100,3.297982,0.686063
3,0.000000,3.319265,0.685849
4,0.000000,3.345392,0.685508
5,0.000000,3.381916,0.684831
6,0.000100,3.388909,0.683458
7,0.034000,3.381931,0.682824


[I 2023-06-16 03:49:02,896] Trial 9 finished with value: 0.6828236101795916 and parameters: {'learning_rate': 1.093274712731568e-06, 'num_train_epochs': 7, 'seed': 9, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 2, 'warmup_steps': 140, 'weight_decay': 0.07056041638210678, 'per_device_eval_batch_size': 16}. Best is trial 6 with value: 0.6835624435402132.


eval/f1,███▇▅▂▁
eval/loss,▁▃▄▆███
eval/runtime,█▃▇▂▆▁▇
eval/samples_per_second,▁▆▂▇▃█▂
eval/steps_per_second,▁▆▂▇▃█▂
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/learning_rate,█▇▆▅▃▂▁
train/loss,▁▁▁▁▁▁█
train/total_flos,▁
train/train_loss,▁


roberta-base chosen hyperparameters:
{'learning_rate': 3.3600626494738905e-06, 'num_train_epochs': 7, 'seed': 9, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 5, 'warmup_steps': 21, 'weight_decay': 0.048934306251516936, 'per_device_eval_batch_size': 16}


## Model 2 Train on best hyperparameters

In [ ]:
best_args = TrainingArguments(output_dir=output_dir,
                         overwrite_output_dir=True,
                         per_device_train_batch_size=16,
                         per_device_eval_batch_size=16,
                         seed = 9,
                         learning_rate=7.600882468235104e-06,
                         weight_decay=0.0511022637864972,
                         greater_is_better=True,
                         evaluation_strategy='epoch',
                         do_train=True,
                         num_train_epochs=20,
                         gradient_accumulation_steps=1,
                         logging_strategy='epoch',
                         warmup_steps=188,
                         report_to='wandb')

eval_args = TrainingArguments(output_dir=output_dir,
                         overwrite_output_dir=True,
                         per_device_train_batch_size=16,
                         per_device_eval_batch_size=16,
                          seed = 9,
                         learning_rate=7.600882468235104e-06,
                         weight_decay=0.0511022637864972,
                         greater_is_better=True,
                         evaluation_strategy='epoch',
                         do_train=False,
                         gradient_accumulation_steps=1,
                         logging_strategy='epoch',
                         warmup_steps=188,
                         report_to='wandb')

roberta.train(best_args)

# save model's state dict
roberta.save_model('roberta_regular')

C:\Users\liyag\AppData\Local\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.011700,3.663433,0.676860
2,0.021200,3.769520,0.674983
3,0.028300,3.862758,0.673112
4,0.035900,3.957393,0.676091
5,0.049400,4.000326,0.676022
6,0.072000,3.980752,0.673859
7,0.184800,3.727416,0.674466
8,0.282000,3.490899,0.676872


RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\caffe2\serialize\inline_container.cc:337] . unexpected pos 860796096 vs 860795984

## Best model summary:

In [ ]:
roberta.model.load_state_dict(torch.load(Path(home_dir, 'best_roberta.py')))
roberta.evaluate(eval_args)
roberta.print_model_size()
roberta.print_model()

## a. Model 2 Pruning

In [ ]:
pruned_roberta = Model(model_name_2, datasets, num_of_classes)
pruned_roberta.tokenize(token_args)

# Load the model's best parameters
pruned_roberta.model.load_state_dict(torch.load(Path(home_dir, 'roberta_regular.py')))

In [ ]:
# Prune and train one epoch
pruned_bert.pruning(0.2)
train_one_epoch_args = TrainingArguments(output_dir=output_dir,
                             overwrite_output_dir=True,
                             greater_is_better=True,
                             evaluation_strategy='epoch',
                             do_train=True,
                             logging_strategy='epoch',
                             num_train_epochs=1,
                             save_strategy='no')

pruned_roberta.train(train_one_epoch_args)
pruned_roberta.save_model('roberta_pruning')

In [ ]:
# Pruned model summary
pruned_roberta.evaluate(eval_args)
pruned_roberta.print_model_size()
pruned_roberta.print_model()

##b.Model 2 Quantization

In [ ]:
# Load the quantized model

quantization_bert = Model(model_name_1, datasets, num_of_classes)
quantization_bert.tokenize(token_args)
quantization_bert.model.load_state_dict(torch.load(Path(home_dir, 'roberta_regular.pt')))

# Train the quantized model for additional epochs
train_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    num_train_epochs=5,  # Specify the desired number of additional epochs
    save_strategy='no'
)

quantization_bert.train(train_args)

# Save the trained quantized model's state dict
quantization_bert.save_model('quantized_reberta.pt')

## c. Model 2 Knowledge-Distillation

In [ ]:
hidden_dim = 128
num_layers = 2
num_heads = 2
num_classes = 62
epochs = 1
alpha_teacher = 0.8
lr=0.001
loss_func = nn.MSELoss()

In [ ]:
kl_roberta = Model(model_name_2, datasets, num_of_classes)
kl_roberta.tokenize(token_args)

# Load the model's best parameters
kl_roberta.model.load_state_dict(torch.load(Path(home_dir, 'best_roberta.py')))

In [ ]:
roberta_student = kl_roberta.kl(hidden_dim,num_layers,num_heads,epochs, alpha_teacher, lr, loss_func)

In [ ]:
roberta_student_model = Model(model_name_1, datasets, num_of_classes)
roberta_student_model.tokenize(token_args)
roberta_student_model.model = roberta_student

roberta_student_model.save_model('roberta_student')

In [ ]:
# Student model summary
roberta_student_model.evaluate(eval_args)
roberta_student_model.print_model_size()
roberta_student_model.print_model()